In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [8]:
data = pd.read_csv('/workspaces/time-series/data/processed/time-serie.csv')
data.head()

,date,sales
0,2022-09-03,55.292157
1,2022-09-04,53.803211
2,2022-09-05,58.141693
3,2022-09-06,64.530899
4,2022-09-07,66.013633


In [9]:
data_train = data.iloc[:-91]
data_test = data.iloc[-91:]

In [10]:
data_train.tail()

,date,sales
270,2023-05-31,753.936865
271,2023-06-01,747.024687
272,2023-06-02,763.812942
273,2023-06-03,761.718225
274,2023-06-04,761.193459


In [11]:
data_test.head()

,date,sales
275,2023-06-05,764.580565
276,2023-06-06,769.837390
277,2023-06-07,770.610592
278,2023-06-08,767.469590
279,2023-06-09,782.357862


In [12]:
n = 4
XY_train = np.array([data_train[i:i+n].sales.values for i in range(len(data_train)) if len(data_train[i:i+n])==n])
X_train, y_train = XY_train[:,:-1], XY_train[:,-1]

XY_test = np.array([data_test[i:i+n].sales.values for i in range(len(data_test)) if len(data_test[i:i+n])==n])
X_test, y_test = XY_test[:,:-1], XY_test[:,-1]

Define el tamaño de la ventana n para el análisis de series de tiempo. En este caso, n = 4.
Para el conjunto de entrenamiento (df_train), crea una matriz XY_train donde cada fila representa una secuencia de longitud n + 1, donde los primeros n elementos son los valores de la columna Average y el último elemento es el valor siguiente de Average. Para hacer esto:
Itera sobre todas las filas del DataFrame df_train utilizando un bucle for.
Para cada fila, extrae una secuencia de longitud n de los valores de la columna Average utilizando la notación de indexación de Pandas [i:i+n].
Si la longitud de la secuencia es igual a n, agrega esta secuencia a la matriz XY_train.
Divide la matriz XY_train en dos partes:
X_train: Contiene todas las columnas excepto la última, que representan los valores de Average en un intervalo de tiempo.
y_train: Contiene la última columna, que representa el valor siguiente de Average.
Repite el mismo proceso para el conjunto de prueba (df_test) para crear las matrices X_test e y_test.
En resumen, este código prepara los datos para un modelo de aprendizaje supervisado donde se intenta predecir el valor siguiente de la columna Average utilizando los valores anteriores en una ventana de tiempo de longitud n.

In [13]:
# Crear el modelo de regresión lineal
X_train_const = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train_const)
# Ajustar el modelo a los datos
results = model.fit()

Añade una columna de unos (constante) a la matriz de características de entrenamiento X_train utilizando sm.add_constant(). Esta columna constante es necesaria para calcular el intercepto en el modelo de regresión lineal.
Crea un modelo de regresión lineal ordinaria (OLS) utilizando la biblioteca statsmodels. Se ajusta el modelo utilizando la función sm.OLS(y_train, X_train_const), donde y_train es la variable de respuesta (la columna Average que se intenta predecir) y X_train_const es la matriz de características de entrenamiento, que incluye la columna constante añadida.

In [14]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.281e+05
Date:                Fri, 03 May 2024   Prob (F-statistic):               0.00
Time:                        11:51:28   Log-Likelihood:                -716.18
No. Observations:                 272   AIC:                             1440.
Df Residuals:                     268   BIC:                             1455.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.3091      0.522     10.161      0.0